In [2]:
import tensorflow as tf
from tensorflow import keras

In [5]:
!pip install -q -U keras-tuner

In [4]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 53.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [6]:
import keras_tuner as kt

In [7]:
(img_train,label_train),(img_test,label_test) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [8]:
#normalizacja danych
img_train =img_train.astype('float32')/255.0
img_test =img_test.astype('float32')/255.0

In [9]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28,28)))

    hp_units = hp.Int('units',min_value=32,max_value=512,step=32)
    model.add(keras.layers.Dense(units=hp_units,activation='relu'))
    model.add(keras.layers.Dense(10))

    hp_learning_rate = hp.Choice('learning_rate',values=[1e-2,1e-3,1e-4])


    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model


In [10]:
#utworzenie wystąpienia tunera i hypertuning
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [11]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5)

In [12]:
#wyszukiwanie hiperparametrów
tuner.search(img_train,label_train,epochs=50,validation_split=0.2,callbacks=[stop_early])
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
        wyszukanie hiparparametrów zakończone.\n
        Optymalna liczba neuronów warstwy ukrytej: {best_hps.get('units')}\n
        Optymalna wartość learning rate: {best_hps.get('learning_rate')}

""")

Trial 30 Complete [00h 00m 52s]
val_accuracy: 0.8914166688919067

Best val_accuracy So Far: 0.8914166688919067
Total elapsed time: 00h 14m 27s

        wyszukanie hiparparametrów zakończone.

        Optymalna liczba neuronów warstwy ukrytej: 384

        Optymalna wartość learning rate: 0.001




In [13]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train,label_train,epochs=50,validation_split=0.2)

val_acc_per_epoch = history.history["val_accuracy"]
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch))+1

print(f'najlepssza epoka: {(best_epoch,)}')

Epoch 1/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.4959 - accuracy: 0.8248 - val_loss: 0.4149 - val_accuracy: 0.8547
Epoch 2/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3719 - accuracy: 0.8653 - val_loss: 0.3540 - val_accuracy: 0.8733
Epoch 3/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3322 - accuracy: 0.8777 - val_loss: 0.3692 - val_accuracy: 0.8682
Epoch 4/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3074 - accuracy: 0.8856 - val_loss: 0.3288 - val_accuracy: 0.8813
Epoch 5/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2870 - accuracy: 0.8949 - val_loss: 0.3319 - val_accuracy: 0.8763
Epoch 6/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2720 - accuracy: 0.8999 - val_loss: 0.3337 - val_accuracy: 0.8822
Epoch 7/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2587 - accuracy: 0.9031 - val_loss: 0.3131 - val_accuracy:

In [14]:
#wykonanie instancji hipermodelu dla optymalnej liczby epok
hypermodel = tuner.hypermodel.build(best_hps)
hypermodel.fit(img_train,label_train,epochs=best_epoch,validation_split=0.2)

Epoch 1/32
1500/1500 [==============================] - 5s 3ms/step - loss: 0.4955 - accuracy: 0.8253 - val_loss: 0.3914 - val_accuracy: 0.8602
Epoch 2/32
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3718 - accuracy: 0.8628 - val_loss: 0.3977 - val_accuracy: 0.8572
Epoch 3/32
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3337 - accuracy: 0.8774 - val_loss: 0.3387 - val_accuracy: 0.8776
Epoch 4/32
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3074 - accuracy: 0.8864 - val_loss: 0.3312 - val_accuracy: 0.8788
Epoch 5/32
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2879 - accuracy: 0.8940 - val_loss: 0.3467 - val_accuracy: 0.8772
Epoch 6/32
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2729 - accuracy: 0.8979 - val_loss: 0.3363 - val_accuracy: 0.8792
Epoch 7/32
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2588 - accuracy: 0.9022 - val_loss: 0.3206 - val_accuracy:

In [15]:
#ocena hipermodelu na danych testowych
eval_results = hypermodel.evaluate(img_test,label_test)
print(f'[test loss, test accuracy]:{eval_results}')

313/313 [==============================] - 1s 2ms/step - loss: 0.4477 - accuracy: 0.8877
[test loss, test accuracy]:[0.4477454721927643, 0.8877000212669373]
